In [3]:
import cv2
import numpy as np


In [4]:
KNOWN_WIDTH = 20.0  # Actual width of the object in cm
KNOWN_DISTANCE = 100.0 

FOCAL_LENGTH = None

# Camera intrinsic parameters (optical center and focal lengths)
cx, cy = 320, 240  # Optical center (assumed at the center of the image)
fx, fy = 800, 800  # Focal lengths (these may need to be calibrated)

In [1]:
# Imports
import cv2

# Define the detection function
def process_video_frame(frame):
    """
    Process a single video frame for object detection.

    Parameters:
    - frame: Input video frame (BGR format).

    Returns:
    - processed_frame: Frame with detection overlays.
    """
    # Example detection logic (replace with your detection code)
    processed_frame = frame.copy()
    cv2.rectangle(processed_frame, (50, 50), (200, 200), (0, 255, 0), 2)  # Draw a dummy bounding box
    cv2.putText(processed_frame, "Detected Object", (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return processed_frame

# Video processing logic
cap = cv2.VideoCapture(0)  # Open webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Use the detection function
    processed_frame = process_video_frame(frame)

    # Display the processed frame
    cv2.imshow("Object Detection", processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/pratham/.local/lib/python3.10/site-packages/cv2/qt/plugins"


KeyboardInterrupt: 

: 

In [5]:
def calculate_focal_length(known_distance, known_width, pixel_width):
    """
    Calculate the camera focal length.
    F = (P * D) / W
    """
    return (pixel_width * known_distance) / known_width

def calculate_distance(focal_length, known_width, pixel_width):
    """
    Calculate the distance to the object.
    D = (W * F) / P
    """
    return (known_width * focal_length) / pixel_width

In [6]:
def measure_pixel_width_with_bounding_box(image, lower_hsv, upper_hsv):
    """
    Measure the pixel width of an object in the image using contours,
    and return the pixel width along with the bounding box coordinates.
    """
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Mask the image to isolate the object based on color
    mask = cv2.inRange(hsv, lower_hsv, upper_hsv)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Assume the largest contour corresponds to the object
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Return pixel width (w) and bounding box coordinates
        return w, (x, y, w, h)
    return None, None

In [7]:
def calculate_3d_coordinates(u, v, depth):
    """
    Calculate the 3D coordinates of the object from its 2D pixel coordinates and depth.
    """
    X = (u - cx) * depth / fx
    Y = (v - cy) * depth / fy
    Z = depth  # The depth itself is the Z-coordinate
    return X, Y, Z

In [8]:
def calculate_3d_coordinates(u, v, depth):
    """
    Calculate the 3D coordinates of the object from its 2D pixel coordinates and depth.
    """
    X = (u - cx) * depth / fx
    Y = (v - cy) * depth / fy
    Z = depth  # The depth itself is the Z-coordinate
    return X, Y, Z

In [ ]:
def main():
    global FOCAL_LENGTH
    cap = cv2.VideoCapture(0)  # Open the camera

    # HSV range for object detection (example: red object)
    lower_hsv = np.array([0, 120, 70])
    upper_hsv = np.array([10, 255, 255])

    # Calibration step
    print("Starting Calibration...")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image.")
            break

        pixel_width, bounding_box = measure_pixel_width_with_bounding_box(frame, lower_hsv, upper_hsv)
        if pixel_width:
            print(f"Pixel Width during Calibration: {pixel_width} px")
            FOCAL_LENGTH = calculate_focal_length(KNOWN_DISTANCE, KNOWN_WIDTH, pixel_width)
            print(f"Calculated Focal Length: {FOCAL_LENGTH:.2f} pixels")

            # Draw bounding box during calibration
            if bounding_box:
                x, y, w, h = bounding_box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, "Calibrating...", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            break

        cv2.imshow("Calibration", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Real-time Depth Estimation
    print("Starting Depth Estimation...")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        pixel_width, bounding_box = measure_pixel_width_with_bounding_box(frame, lower_hsv, upper_hsv)
        if pixel_width:
            distance = calculate_distance(FOCAL_LENGTH, KNOWN_WIDTH, pixel_width)
            print(f"Distance to Object: {distance:.2f} cm")

            # Draw bounding box and display distance
            if bounding_box:
                x, y, w, h = bounding_box
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, f"Distance: {distance:.2f} cm", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        cv2.imshow("Depth Estimation", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        pixel_width, bounding_box = measure_pixel_width_with_bounding_box(frame, lower_hsv, upper_hsv)
        if pixel_width:
            depth = calculate_distance(FOCAL_LENGTH, KNOWN_WIDTH, pixel_width)
            print(f"Distance to Object: {depth:.2f} cm")

            # Get the 3D coordinates
            x, y, w, h = bounding_box
            u, v = x + w // 2, y + h // 2  # Center of the bounding box
            X_3d, Y_3d, Z_3d = calculate_3d_coordinates(u, v, depth)
            print(f"3D Coordinates of Object: X = {X_3d:.2f} cm, Y = {Y_3d:.2f} cm, Z = {Z_3d:.2f} cm")

            # Draw bounding box and display distance and 3D coordinates
            if bounding_box:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, f"Distance: {depth:.2f} cm", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
                cv2.putText(frame, f"3D: ({X_3d:.2f}, {Y_3d:.2f}, {Z_3d:.2f})", (x, y - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        cv2.imshow("Depth and 3D Estimation", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [ ]:
def detect_color(image, lower_hsv, upper_hsv):
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Create a binary mask where the detected color is white
    mask = cv2.inRange(hsv_image, lower_hsv, upper_hsv)
    
    # Apply dilation to fill gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))  # Kernel size can be adjusted
    dilated_mask = cv2.dilate(mask, kernel, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours, dilated_mask

In [ ]:
# Adjust these values based on your target color
lower_hsv = np.array([0, 120, 70])  # Lower bound of HSV (red)
upper_hsv = np.array([10, 255, 255])  # Upper bound of HSV (red)

In [ ]:
cap = cv2.VideoCapture(0)  

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Blur the image to reduce noise
    blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)
    
    # Detect the target color
    contours, processed_mask = detect_color(blurred_frame, lower_hsv, upper_hsv)
    
    # Draw contours on the original frame
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Ignore small contours
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, "Color Detected", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the processed mask and the original frame
    cv2.imshow("Mask", processed_mask)
    cv2.imshow("Detected Color", frame)
    
    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

"Steps in Color Recognition:
Turn on the Camera and Capture the Image:

The system begins by capturing the scene using a camera.
Image Blurring:

The captured image is blurred (e.g., using Gaussian Blur) to reduce noise and smooth the image.
This step enhances the accuracy of subsequent processing by minimizing minor variations.
Color Identification Using HSV Values:

The image is converted from the RGB color space to the HSV (Hue, Saturation, Value) color space.
Hue: Represents the type of color (e.g., red, green, blue).
Saturation: Intensity or purity of the color.
Value: Brightness of the color.
Specific ranges of HSV values are used to detect the desired colors.
Thresholding and Dilation:

Thresholding: Converts the image into a binary mask based on HSV ranges, isolating pixels of interest.
Dilation: Expands the detected color regions to fill in gaps or imperfections, making the features clearer.
Avoiding Erosion:

Erosion, which reduces pixel regions, is avoided in this procedure because it might introduce gaps in extracted lines.
Instead, only dilation is performed to ensure continuity in the detected lines.
Contour Detection:

Contours are identified based on the detected regions of interest.
The contours are framed or marked to visualize and outline the detected color regions.